In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 1e18 + 7777
def sol() :
  N, M = map(int, input().split())
  G = [[] for _ in range(N + M + 1)]
  for i in range(1, M+1) :
    u, v, w = map(int, input().split())
    G[u].append((N+i, w / 2))
    G[v].append((N+i, w / 2))
    G[N+i].append((u, w / 2))
    G[N+i].append((v, w / 2))

  ans = INF
  for s in range(1, N + 1) :
    Q, nextQ = [(0, s)], []
    vis = [INF] * (N + M + 1)
    vis[s] = 0
    res = 0
    while Q :
      while Q :
        uw, u = Q.pop()
        vis[u] = uw
        res = max(res, uw)
        for v, vw in G[u] :
          if vis[v] : continue
          nextQ.append((uw + vw, v))
      Q, nextQ = nextQ, []
      nextQ.clear()
    ans = min(ans, res)
  print(ans)
sol()

- 약간 잘못된 접근. 한쪽 간선이 매우 길고, 나머지 간선이 1인 사이클을 이루는 그래프를 생각해보자. 
  - 100쪽 간선을 바로 태워버리고 그다음 1 짜리 다음 간선을 바로 태워버린다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def sol() :
  N, M = map(int, input().split())
  E = []
  acc = 0 
  for _ in range(M) :
    u, v, w = map(int, input().split())
    acc += w - 1
    E.append((u, v, w))

  G = [[] for _ in range(N + acc + 1)]
  idx = N + 1
  for u, v, w in E :
    for _ in range(w-1) :
      G[u].append(idx)
      G[idx].append(u)
      u = idx
      idx += 1
    G[u].append(v)
    G[v].append(u)

  ans = INF
  for s in range(1, N + 1) :
    time = 0
    DP = [-1] * (N + acc + 1) #불타 없어진 시간
    DP[s] = 0
    vis = [False] * (N + acc + 1)
    vis[s] = True
    Q, nextQ = [s], []
    while Q :
      flag = False #0.5초를 더해줘야 하는지 여부
      while Q :
        u = Q.pop()
        DP[u] = time
        for v in G[u] :
          if DP[v] == time: flag = True #양쪽의 정점은 이미 탔고, 양쪽에서 서로 간선을 태우기 시작한 경우
          if vis[v] : continue
          vis[v] = True
          nextQ.append(v)
      Q, nextQ = nextQ, Q
      time += 1
    res = time + flag * 0.5 - 1
    ans = min(ans, res)
  print(ans)
sol()

- 메모리 제한이 특별하게 적은걸 발견했다.
  - 이렇게 된이상 bfs마다 정점 추가와 간선 가중치 감소를 직접 시뮬레이팅 할 수 밖에 없겠다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def sol() :
  N, M = map(int, input().split())
  E = []
  for _ in range(M) :
    u, v, w = map(int, input().split())
    E.append((u, v, w))

  ans = INF
  for s in range(1, N + 1) :
    G = [[] for _ in range(N + 1)]
    W = []
    for id, (u, v, w) in enumerate(E) :
      W.append(w)
      G[u].append((id, v))
      G[v].append((id, u))

    Q, nextQ = [], []
    DP = [-1] * M #간선이 불타 없어진 시점
    while G[s] :
      id, u = G[s].pop()
      Q.append((id, u))

    flag = False
    t = 0
    while Q :
      flag = False
      while Q :
        id, u = Q.pop()
        W[id] -= 1
        if W[id] == -1 : 
          flag |= DP[id] == t
        elif W[id] == 0 :
          DP[id] = t
          while G[u] :
            id, v = G[u].pop()
            nextQ.append((id, v))
        else :
          nextQ.append((id, u))
      
      Q, nextQ = nextQ, Q
      t += 1
    ans = min(ans, t + flag * 0.5 - 1)
  print(ans)
sol()

- 이번엔 시간초과. N번 돌려보는게 아닌가? 또 다른 그래프 순회 이슈인가?

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def sol() :
  N, M = map(int, input().split())
  E = []
  for _ in range(M) :
    u, v, w = map(int, input().split())
    E.append((u, v, w))

  ans = INF
  for s in range(1, N + 1) :
    G = [[] for _ in range(N + 1)]
    W = []
    for id, (u, v, w) in enumerate(E) :
      W.append(w)
      G[u].append((id, v))
      G[v].append((id, u))

    Q, nextQ = [], []
    DP = [-1] * M #간선이 불타 없어진 시점
    while G[s] :
      id, u = G[s].pop()
      Q.append((id, u))

    flag = False
    t = 0
    while Q :
      flag = False
      while Q :
        uid, u = Q.pop()
        W[uid] -= 1
        if W[uid] == -1 :
          flag |= DP[uid] == t
          continue

        if W[uid] == 0 : 
          DP[uid] = t
          while G[u] :
            vid, v = G[u].pop()
            nextQ.append((vid, v))
          continue
        
        nextQ.append((uid, u))
      
      Q, nextQ = nextQ, Q
      t += 1
    ans = min(ans, t + flag * 0.5 - 1)
  print(ans)
sol()

- 그래프 이슈 문제가 아니다. 아무래도 N번 돌리지 않아도 알 수 있는 것 같다.
- $N \leq 200$에서 생각해보건데, 플로이드 와샬로 모든 정점까지의 거리가 가장 짧은 정점에서 시도하면 되지 않을까?

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def FW(V: int, E: list):
  D = [[0 if u == v else INF for u in range(V)] for v in range(V)]

  for u, v, w in E:
    D[u][v] = w
    D[v][u] = w

  for m in range(V):
    for u, l in enumerate(D):
      for v, w in enumerate(l):
        if l[m] + D[m][v] >= w: continue
        l[v] = l[m] + D[m][v]

  return D

def sol() :
  N, M = map(int, input().split())
  E = []
  self_dist = [0] * (N + 1)
  for _ in range(M) :
    u, v, w = map(int, input().split())
    E.append((u, v, w))
    if u == v : self_dist[u] = max(self_dist[u], w)
  
  D = FW(N + 1, E)
  s = -1
  x = INF
  for i, l in enumerate(D[1:], 1) :
    acc = -self_dist[i]
    for v in l :
      if v == INF : continue
      acc += v
    if acc < x :
      x = acc
      s = i

  G = [[] for _ in range(N + 1)]
  W = []
  for id, (u, v, w) in enumerate(E) :
    W.append(w)
    G[u].append((id, v))
    G[v].append((id, u))

  ans = INF
  Q, nextQ = [], []
  DP = [-1] * M #간선이 불타 없어진 시점
  while G[s] :
    id, u = G[s].pop()
    Q.append((id, u))

  flag = False
  t = 0
  while Q :
    flag = False
    while Q :
      uid, u = Q.pop()
      W[uid] -= 1
      if W[uid] == -1 :
        flag |= DP[uid] == t
        continue

      if W[uid] == 0 : 
        DP[uid] = t
        while G[u] :
          vid, v = G[u].pop()
          nextQ.append((vid, v))
        continue
      
      nextQ.append((uid, u))
    
    Q, nextQ = nextQ, Q
    t += 1
  ans = min(ans, t + flag * 0.5 - 1)
  print(ans)
sol()

- WA.. 이젠 모르겟어...

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def FW(V: int, E: list):
  D = [[0 if u == v else INF for u in range(V)] for v in range(V)]

  for u, v, w in E:
    D[u][v] = D[v][u] = min(D[v][u], w)

  for m in range(V):
    for u, l in enumerate(D):
      for v, w in enumerate(l):
        if l[m] + D[m][v] >= w: continue
        l[v] = l[m] + D[m][v]

  return D

def sol() :
  N, M = map(int, input().split())
  E = []
  self_dist = [0] * (N + 1)
  for _ in range(M) :
    u, v, w = map(int, input().split())
    E.append((u, v, w))
    if u == v : self_dist[u] = max(self_dist[u], w)
  
  D = FW(N + 1, E)
  s = -1
  x = INF
  for i, l in enumerate(D[1:], 1) :
    acc = -self_dist[i]
    for j, v in enumerate(l) :
      if v == INF : continue
      if i == j : continue
      acc += v
    if acc < x :
      x = acc
      s = i

  G = [[] for _ in range(N + 1)]
  W = []
  for id, (u, v, w) in enumerate(E) :
    W.append(w)
    G[u].append((id, v))
    G[v].append((id, u))

  ans = INF
  Q, nextQ = [], []
  DP = [-1] * M #간선이 불타 없어진 시점
  while G[s] :
    id, u = G[s].pop()
    Q.append((id, u))

  flag = False
  t = 0
  while Q :
    flag = False
    while Q :
      uid, u = Q.pop()
      W[uid] -= 1
      if W[uid] == -1 :
        flag |= DP[uid] == t
        continue

      if W[uid] == 0 : 
        DP[uid] = t
        while G[u] :
          vid, v = G[u].pop()
          nextQ.append((vid, v))
        continue
      
      nextQ.append((uid, u))
    
    Q, nextQ = nextQ, Q
    t += 1
  ans = min(ans, t + flag * 0.5 - 1)
  print(ans)
sol()

- 플로이드 워셜을 생각해보니 다른 관점이 생각난다.
  - 각 정점에 도착하고나서부터 간선을 태운다고 볼 수 있다. 
  - 간선이 가장 늦게 타는 시점을 계산할 수 있을지도 모른다.
    - 근데 양쪽에서 동시에 타는 판정 계산은 너무 어려운 것 같다..
- 자기 자신의 거리가 긴걸 고르는게 이득은 맞다.
- 자기 자신의 정점과 연결된건 2배속으로 탄다.
- 좀더 정리 해보자. $u, v$ 로 연결되는 간선을 태우는 속도는 
  - $u$ 에 도착한 시간과 $v$ 에 도착한 시간과 연관이 있다.
  - 구체적으로, $u$ 에 도착한 순간 1배속으로 타다가, $v$ 에 도착한 순간 2배속으로 타게 된다.
  - 문제의 정답은, 점화 이후 가장 늦게 도달한 간선이 모두 타는데 걸리는 시간과 같다.
  - 정점 $u, v$ 사이에 연결된 길이 $y$ 의 간선 $e$ 가 타기 시작하는 시간을 $t_a$, 2배속으로 타기 시작하는 시간을 $t_b$ 라고 해보자.
    - 어떤 정점 $x$ 로 부터 그래프의 점화가 시작되었다고 하고, $u, v$ 중 $x$ 와의 거리가 먼 정점을 $v$ 라고 하자.\
    $t_a$ 는 $x, u$ 사이의 거리이다. 그 시점에 $e$ 의 1배속 연소가 시작된다.\
    $t_b$ 는 $x, v$ 사이의 거리와 같다. 그 시점에 $e$ 의 2배속 연소가 시작된다.\
    $t_a$ 에서 $e$ 의 남은 길이는 $y - (t_b - t_a)$ 이다.\
    그리고 2배속 연소가 시작된 시점부터 그 절반만큼의 시간이 지나면 $e$ 는 연소가 끝난다. \
    즉, 남은 길이의 절반만큼의 시간이 소모된다. $\frac{y - (t_b - t_a)}{2}$\
    그러한 점화는 $t_b$ 부터 시작되게 된다. 따라서 $t_b + \frac{y - (t_b - t_a)}{2}$ 가 $e$ 가 전소되는 시간이다.
    - 정점 사이의 거리를 모두 구한다면, 한 간선의 연소시간을 $O(1)$ 에 구할 수 있다.
    - 이정도면 $O(NM)$ 으로 충분히 구할 수 있다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
def FW(V: int, E: list):
  D = [[0 if u == v else INF for u in range(V)] for v in range(V)]

  for u, v, w in E:
    D[u][v] = D[v][u] = min(D[v][u], D[u][v], w)

  for m in range(V):
    for u, l in enumerate(D):
      for v, w in enumerate(l):
        if l[m] + D[m][v] >= w: continue
        l[v] = l[m] + D[m][v]

  return D

def sol() :
  N, M = map(int, input().split())
  E = []
  for _ in range(M) :
    u, v, w = map(int, input().split())
    E.append((u, v, w))
  
  D = FW(N + 1, E)
  D[0] = [INF] * (N + 1)
  ans = INF
  for s in range(1, N+1) : 
    res = 0 #s를 점화지점으로 선택했을 때, 가장 늦게 전소되는 간선의 전소시각
    for u, v, w in E :
      a, b = D[s][u], D[s][v]
      if a > b :
        a, b = b, a 
      res = max(res, b + (w - (b - a)) / 2)
    ans = min(ans, res)
  print(ans)
  
sol()

5 8
1 2 4
1 2 6
1 5 6
2 4 4
4 5 4
3 4 6
3 4 6
3 3 4
;;
5 10
1 2 1
2 3 1
3 4 1
4 5 1
1 3 10
2 4 10
3 5 10
1 4 7
2 5 7
1 5 9
;; 2.5
3 4
1 1 5
1 2 1
2 3 1
1 3 1
;; 2.0
3 4
1 1 4
1 2 1
2 3 1
1 3 1
;; 0.5
1 1
1 1 1
;; 1.0
1 1
1 1 2
;;
2 4
1 1 2
2 2 20003
1 2 10000
1 2 10000

### 자력솔(with tag hint)
- 힘들었다.. 이렇게 고생하면서 문제를 잡고 고민한적이 없다. 시간도 오래걸리고.